<a href="https://colab.research.google.com/github/JustMatthew989/Matvey_Mokritskikh_Repo/blob/main/%D0%94%D0%97_%D0%9C%D0%B5%D1%82%D0%BE%D0%B4%D0%9E%D0%BF%D0%BE%D1%80%D0%BD%D1%8B%D1%85%D0%92%D0%B5%D0%BA%D1%82%D0%BE%D1%80%D0%BE%D0%B2_%D0%9C%D0%BE%D0%BA%D1%80%D0%B8%D1%86%D0%BA%D0%B8%D1%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#ДОМАШНЕЕ ЗАДАНИЕ №5

In [ ]:
#ЗАДАНИЕ №1

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [ ]:
df = pd.DataFrame(pd.read_csv(r'C:\Users\snady\.spyder-py3\laptop_prices.csv'))

for column in df.columns:
    if df[column].dtype in ['int64', 'float64']:
        df[column].fillna(0)
    else:
        df[column].fillna('other')

df.drop_duplicates(inplace=True)

for column in df.columns:
    if df[column].nunique() == len(df):
        df.drop(column, axis=1, inplace=True)

categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

ohe = OneHotEncoder(drop='first', sparse_output=False)
encoded_data = pd.DataFrame(
    ohe.fit_transform(df[categorical_columns]),
    columns=ohe.get_feature_names_out(categorical_columns)
)

df = pd.concat([df[numerical_columns], encoded_data], axis=1)

scaler = StandardScaler()
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

if 'Price_euros' in df.columns:
    X = df.drop('Price_euros', axis=1)
    y = df['Price_euros']
else:
    raise ValueError("Целевой столбец 'Price_euros' отсутствует в данных.")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#ЗАДАНИЕ №2

In [ ]:
model = SVR()
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_r2 = r2_score(y_train, y_train_pred)
train_mse = mean_squared_error(y_train, y_train_pred)
train_mae = mean_absolute_error(y_train, y_train_pred)

test_r2 = r2_score(y_test, y_test_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

print("Метрики train:")
print(f"R2: {train_r2:.4f}")
print(f"MSE: {train_mse:.4f}")
print(f"MAE: {train_mae:.4f}\n")

print("Метрики test:")
print(f"R2: {test_r2:.4f}")
print(f"MSE: {test_mse:.4f}")
print(f"MAE: {test_mae:.4f}\n")

Метрики train:
R2: 0.9099
MSE: 0.0897
MAE: 0.1538

Метрики test:
R2: 0.8122
MSE: 0.1900
MAE: 0.2614



In [ ]:
#ЗАДАНИЕ №3

In [ ]:
import pandas as pd
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

df = pd.DataFrame(pd.read_csv(r'C:\Users\snady\.spyder-py3\laptop_prices.csv'))

for column in df.columns:
    if df[column].dtype in ['int64', 'float64']:
        df[column] = df[column].fillna(0)
    else:
        df[column] = df[column].fillna('other')

df.drop_duplicates(inplace=True)

for column in df.columns:
    if df[column].nunique() == len(df):
        df.drop(column, axis=1, inplace=True)

categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

ohe = OneHotEncoder(drop='first', sparse_output=False)
encoded_data = pd.DataFrame(
    ohe.fit_transform(df[categorical_columns]),
    columns=ohe.get_feature_names_out(categorical_columns)
)

df = pd.concat([df[numerical_columns], encoded_data], axis=1)

scaler = StandardScaler()
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

if 'Price_euros' in df.columns:
    X = df.drop('Price_euros', axis=1)
    y = df['Price_euros']
else:
    raise ValueError("Целевой столбец 'Price_euros' отсутствует в данных.")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Это функция для оптуны
def objective(trial):
    kernel = trial.suggest_categorical('kernel', ['poly', 'rbf'])
    C = trial.suggest_float('C', 1, 100, log=True)
    gamma = trial.suggest_float('gamma', 0.01, 100, log=True)

    model = SVR(kernel=kernel, C=C, gamma=gamma)
    model.fit(X_train, y_train)


    y_pred = model.predict(X_test)


    r2 = r2_score(y_test, y_pred)
    return r2

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print("Best hyperparameters: ", study.best_params)

best_params = study.best_params
model = SVR(kernel=best_params['kernel'], C=best_params['C'], gamma=best_params['gamma'])
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_r2 = r2_score(y_train, y_train_pred)
train_mse = mean_squared_error(y_train, y_train_pred)
train_mae = mean_absolute_error(y_train, y_train_pred)

test_r2 = r2_score(y_test, y_test_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

print("Метрики train:")
print(f"R2: {train_r2:.4f}")
print(f"MSE: {train_mse:.4f}")
print(f"MAE: {train_mae:.4f}\n")

print("Метрики test:")
print(f"R2: {test_r2:.4f}")
print(f"MSE: {test_mse:.4f}")
print(f"MAE: {test_mae:.4f}\n")

[I 2024-12-24 18:53:08,172] A new study created in memory with name: no-name-0bc99151-f48f-46c1-b43e-b3a2f66afd0a
[I 2024-12-24 18:53:08,635] Trial 0 finished with value: 0.8836430482868382 and parameters: {'kernel': 'poly', 'C': 1.7338101040042357, 'gamma': 16.129762538240644}. Best is trial 0 with value: 0.8836430482868382.
[I 2024-12-24 18:53:09,245] Trial 1 finished with value: 0.016971389781442525 and parameters: {'kernel': 'rbf', 'C': 1.3742520123580482, 'gamma': 2.527603408961121}. Best is trial 0 with value: 0.8836430482868382.
[I 2024-12-24 18:53:09,670] Trial 2 finished with value: 0.8742908199115416 and parameters: {'kernel': 'rbf', 'C': 5.891740297675504, 'gamma': 0.011249184628453406}. Best is trial 0 with value: 0.8836430482868382.
[I 2024-12-24 18:53:10,252] Trial 3 finished with value: 0.02381484716365112 and parameters: {'kernel': 'rbf', 'C': 78.14895873339023, 'gamma': 4.771652639139567}. Best is trial 0 with value: 0.8836430482868382.
[I 2024-12-24 18:53:10,753] Tria

Best hyperparameters:  {'kernel': 'rbf', 'C': 15.323595084278862, 'gamma': 0.016473745470331813}
Метрики train:
R2: 0.9797
MSE: 0.0202
MAE: 0.0984

Метрики test:
R2: 0.9088
MSE: 0.0923
MAE: 0.2227



Вывод: Использование Optuna для подбора гиперпараметров в значительной степени улучшило метрики и на тренировочной, и на тествыборке. Действительно нетрудно это заметить, если взглянуть на показатели R2, MSE, и MAE. Подводя итог, модель после оптимизации лучше находит правильные параметры и, соответственно, выдает более точные предикты.